## Setup Dataset

In [ ]:
from src.data.setup import download_and_extract

download_and_extract(
    url="https://drive.google.com/uc?id=1VAc_MFUAlrtqZanxwYRRtGYVLFunKABf",
    zip_path="data.zip",
    extract_dir="data",
    verbose=True,
    remove_zip=True
)

## Offline-Train All Classifiers

In [1]:
from src.models.decision_tree.train import main as train_decision_tree
from src.models.hoeffding.train import main as train_hoeffding_tree
from src.models.weighted_forest.train import main as train_weighted_forest

train_decision_tree()
train_hoeffding_tree()
train_weighted_forest()

Running offline training for Decision Tree
Loading data...
Training Decision Tree...
Evaluating model...
  Training Accuracy: 0.9404
  Test Accuracy: 0.8928
Saving model...
  Model saved to: models/dt/decision_tree_pong.pkl
  Metadata saved to: models/dt/decision_tree_metadata.pkl
Training Finished for Decision Tree

Running offline training for Hoeffding Tree
Loading data...
Training Hoeffding Tree...
  Progressive validation accuracy: 0.6255
Evaluating model...
  Training Accuracy: 0.6255
  Test Accuracy: 0.6582
Saving model...
  Model saved to: models/ht/hoeffding_tree_pong.pkl
  Metadata saved to: models/ht/hoeffding_tree_metadata.pkl
Training Finished for Hoeffding Tree

Running offline training for Weighted Forest
Loading data...
Training Weighted Forest...


/Users/julian/workspace/pwr/ml/mlproj/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julian/workspace/pwr/ml/mlproj/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


1 Num cells: 0
2 Num cells: 0
3 Num cells: 2
4 Num cells: 3
5 Num cells: 2
Evaluating model...
  Training Accuracy: 0.3156
  Test Accuracy: 0.3107
Saving model...
  Model saved to: models/wf/weighted_forest_pong.pkl
  Metadata saved to: models/wf/weighted_forest_metadata.pkl
Training Finished for Weighted Forest



## Launch Interactive Game

Left player controls the paddle using Q (up) and A (down).

Choose the enemy using the mode variable. Choices include:
- "human": play against another player
- "pc": pc player aiming to reach ball_y
- "dt": offline-only trained decision tree
- "ht": offline-only pre-trained hoeffding tree
- "wf:" offline-only pre-trained weighted forest

In [ ]:
from src.main import main

mode = "ht" # "human", "pc", "dt", "ht", "ct"
main(mode)

## Evaluate All Offline Pre-Trained Models

Left paddle will be controlled by PC player.

Evaluation metric is the amount of ball collisions by the tested model and final game score after max-score was reached by one player.

In [ ]:
from src.evaluation import evaluate_all_models

results = evaluate_all_models(max_score=3)

for model_name, result in results.items():
    print(f"{model_name.upper()}: {result.survival_time_seconds:.2f}s, "
          f"Score {result.final_pc_score}-{result.final_ai_score}, "
          f"Hits: {result.total_hits}")

## Online Training - Hoeffding Tree

In [ ]:
from src.training.train_online import train_hoeffding_online
from src.training.train_online import train_weighted_forest_online

train_hoeffding_online(
    pretrained_model_path="models/ht/hoeffding_tree_pong.pkl",
    num_episodes=2,
    max_score_per_episode=3,
    save_interval=5
)

train_weighted_forest_online(
    pretrained_model_path="models/wf/weighted_forest_pong.pkl",
    metadata_path="models/wf/weighted_forest_metadata.pkl",
    num_episodes=2,
    max_score_per_episode=3,
    save_interval=5
)

## Compare Pretrained vs Online Models

In [ ]:
from src.evaluation import compare_pretrained_vs_online

results = compare_pretrained_vs_online(max_score=5)

## Visualize Online Training Metrics

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ht_metrics = pd.read_csv("models/ht/hoeffding_online_metrics.csv")
wf_metrics = pd.read_csv("models/wf/weighted_forest_online_metrics.csv")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].plot(ht_metrics['episode'], ht_metrics['survival_seconds'], marker='o')
axes[0, 0].set_title('Hoeffding Tree - Survival Time')
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Survival (seconds)')

axes[0, 1].plot(ht_metrics['episode'], ht_metrics['progressive_accuracy'], marker='o')
axes[0, 1].set_title('Hoeffding Tree - Progressive Accuracy')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Accuracy')

axes[1, 0].plot(wf_metrics['episode'], wf_metrics['survival_seconds'], marker='o')
axes[1, 0].set_title('Weighted Forest - Survival Time')
axes[1, 0].set_xlabel('Episode')
axes[1, 0].set_ylabel('Survival (seconds)')

axes[1, 1].plot(wf_metrics['episode'], wf_metrics['num_cells'], marker='o')
axes[1, 1].set_title('Weighted Forest - Active Cells')
axes[1, 1].set_xlabel('Episode')
axes[1, 1].set_ylabel('Number of Cells')

plt.tight_layout()
plt.show()